In [ ]:
# Health Insurance Premium Prediction Project

## Project Overview
This project aims to predict health insurance premiums using Ordinary Least Squares (OLS) regression with comprehensive feature engineering and selection techniques including:
- Variance Inflation Factor (VIF) analysis for multicollinearity detection
- Correlation analysis for feature selection
- Feature engineering to improve model performance

## Dataset
The dataset contains health insurance information with features like age, sex, BMI, children, smoker status, region, and insurance charges.

## Methodology
1. **Data Exploration and Preprocessing**
2. **Feature Engineering**
3. **Feature Selection using Correlation and VIF**
4. **OLS Model Implementation**
5. **Model Evaluation and Diagnostics**

In [ ]:
# Import necessary libraries for data manipulation, visualization, and modeling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the insurance dataset
df = pd.read_csv('insurance.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())